# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
# number of callbacks for white-sounding names
sum(data[data.race=='w'].call)

235.0

In [6]:
data.race.value_counts()

b    2435
w    2435
Name: race, dtype: int64

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1,0,1,0,0,0,0,0,0,
1,b,1,3,3,6,0,1,1,0,316,...,1,0,1,0,0,0,0,0,0,
2,b,1,4,1,6,0,0,0,0,19,...,1,0,1,0,0,0,0,0,0,
3,b,1,3,4,6,0,1,0,1,313,...,1,0,1,0,0,0,0,0,0,
4,b,1,3,3,22,0,0,0,0,313,...,1,1,0,0,0,0,0,1,0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

### Question 1: What test is appropriate for this problem? Does CLT apply?

An appropriate test would be the two-sample test of difference between proportions. CLT applies since the following condition is met for each race category:
  
  
** White race:**   
- n.p_hat = 235 >= 10  
- n.(1-p_hat) = 2,200 >= 10     

** Black race:**   
- n.p_hat = 157 >= 10   
- n.(1-p_hat) = 2,278 >= 10

### Question 2: What are the null and alternate hypotheses?

**H0:** The call back proportion for white and black races are equal  
**Ha:** There is a difference between the call back proportion between the white and black races

### Question 3: Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.


In [4]:
w = data[data.race=='w']
b = data[data.race=='b']

** 95% CI for difference in proportions using frequentist approach **:

In [6]:
## Sample proportion for white names
p_hat_w=np.sum(w.call)/len(w.call)
p_hat_w

0.096509240246406572

In [5]:
## Sample proportion for black names
p_hat_b=np.sum(b.call)/len(b.call)
p_hat_b

0.064476386036960986

In [7]:
## Difference between sample proportions
p_hat_w - p_hat_b

0.032032854209445585

In [24]:
## Margin of error
margin_of_error = 1.96*np.sqrt(p_hat_w*(1-p_hat_w)/len(w.call)+p_hat_b*(1-p_hat_b)/len(b.call))
margin_of_error

0.015255406349886438

In [25]:
## 95% CI upper bound
CI_upper = p_hat_w-p_hat_b + margin_of_error
CI_upper

0.047288260559332024

In [26]:
## 95% CI lower bound
CI_lower = p_hat_w-p_hat_b - margin_of_error
CI_lower

0.016777447859559147

** Conclusion:** This 95% CI does not include zero. We conclude that there is a difference between call back rate between white and black races (the rate for white is higher) 

** Hypothesis testing using frequentist approach: **

**H0:** The call back proportion for white and black races are equal  
**Ha:** There is a difference between the call back proportion between the white and black races

In [29]:
## Proportion of total call back in the sample
p_hat_all = np.sum(data.call)/len(data.call)
p_hat_all

0.080492813141683772

In [33]:
## Calculating the z statistic
z = (p_hat_w - p_hat_b - 0)/np.sqrt(2*p_hat_all*(1-p_hat_all)/(len(data.call)))
z

5.8101721857910924

In [41]:
p_value = 1 - stats.norm(0,1).cdf(5.81)
p_value

3.1236421227376354e-09

** Conclusion: ** The P_value is small (i.e. < 5%). The conclusion is that there is difference bwteen the call back rate between the two races

** 95% CI for difference in proportions using bootstrap method **:

In [43]:
## Defining a function that calculates the proportion of interest from a bootstrap sample
def bootstrap_replicate_prop(var):
    """Generate proportion of interest for a bootstrap sample."""
    bs_sample = np.random.choice(var, len(var))
    prop = np.sum(bs_sample)/len(bs_sample)
    return prop

In [46]:
## Initializing empty arrays of length 10000
bs_w_replicates=np.empty(10000)
bs_b_replicates=np.empty(10000)
bs_replicates_diff=np.empty(10000)

In [47]:
# Bootstrap sampling and putting the differences of sample proportions in a numpy array
for i in range(10000):
    bs_w_replicates[i]=bootstrap_replicate_prop(w.call)
    bs_b_replicates[i]=bootstrap_replicate_prop(b.call)
    bs_replicates_diff[i]=bs_w_replicates[i]-bs_b_replicates[i]


In [49]:
## Calculating the 95% CI
CI=np.percentile(bs_replicates_diff,[2.5,97.5])
CI

array([ 0.0164271 ,  0.04722793])

** Conclusion **: This is, almost, the same 95% CI as the one we got using the frequentist approach. The conclusion would be the same as before (i.e. there is a difference in the rate of call back for between white and black races)

** Hypothesis testing using bootstrap method: **

In [14]:
## Initializing an empty array to hold permutation sample proportion differences 
perm_sample_prop_diff=np.empty(10000)

## Producing 10,000 permutation samples under the null hypothesis and calculating their proportion differences
for i in range(10000):
    calls_perm = np.random.permutation(data.call) ## Permuting the original data
    perm_sample_prop_w = np.mean(calls_perm[:len(w.call)]) ## Calling the first half white
    perm_sample_prop_b = np.mean(calls_perm[len(w.call):]) ## Calling the second half black
    perm_sample_prop_diff[i] = perm_sample_prop_w - perm_sample_prop_b 

In [10]:
p_value = np.sum(perm_sample_prop_diff >= p_hat_w - p_hat_b)/len(perm_sample_prop_diff)
p_value

0

** Conclusion: ** As we concluded in previous tests, because of the small p_value, we reject the null and conclude that there is difference in call back rate between black and white races

### Question 4: Write a story describing the statistical significance in the context of the original problem:

To see if there is a difference between the call back rate from employers between black and white races, 4,780 resumes were labeled with white-sounding and black-sounding names and sent to employers. In this project we tested whether there is a difference between the call back rate between the black-sounding and white-sounding names/groups. The range of values for the calculated 95% confidence interval for the difference of proportions between the two groups was higher than zero (using both frequentist an bootstrap methods). So we are 95% confident that the mean diference between proportions is higher than zero. This means the call back rate for the white race/names is higher. The hypothesis tests yielded the same results.  

### Question 5: Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, the analysis is not comparing between different factors that impact the call back rate. This analysis is only focused on whether race has an impact in whether or not the applicant received a call back. There might be other factors that are more effective in explaining racist behavior in call backs. To see if that is the case we need to perform multivariate analyses including potential factors other than race.  